# Data analyse en processing
In dit notebook gaan wij kijken naar de sample set die gegenereerd is in een eerder notebook (`1 - sample`) en hoe wij hiermee verder kunnen gaan.

## Libraries importeren

In [184]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

## Data importeren

De data die gebruikt wordt is de data die gegenereerd is in het notebook `1 - sample`.

In [185]:
path = 'data/'
df = pd.read_csv(path + 'sample.csv')

In het notebook `1 - sample` zijn de eerste 6 maanden en de laatste 6 maanden van 2020 samengevoegd tot een sample set. Bij het mergen van deze twee sets is er een kolom `Unnamed: 0` toegevoegd aan de sample set. Deze is onnodig voor ons en kan er dus uitgehaald worden.

In [186]:
df = df.drop(columns=['Unnamed: 0'])

## Data valideren
Om te kijken of de sample set correct geprocessed is, gaan wij deze eerst valideren.

Ten eerste kijken wij naar het begin van de dataframe, middels de functie `head()`, en naar het einde van de dataframe, middels de functie `tail()`. Hiermee valideren wij dat de data begint op 1 januari 2020 (2020-01-01) en eindigt op 31 december 2020 (2020-12-31).

In [187]:
df.head()

,date,serial_number,model,failure,smart_5_raw,smart_9_raw,smart_187_raw,smart_188_raw,smart_194_raw,smart_197_raw,smart_198_raw
0,2020-01-01,Z305B2QN,ST4000DM000,0,0.0,35462.0,0.0,0.0,22.0,0.0,0.0
1,2020-01-01,ZJV0XJQ4,ST12000NM0007,0,0.0,12494.0,0.0,0.0,28.0,0.0,0.0
2,2020-01-01,ZJV0XJQ3,ST12000NM0007,0,0.0,9544.0,0.0,0.0,29.0,0.0,0.0
3,2020-01-01,ZJV0XJQ0,ST12000NM0007,0,0.0,13098.0,0.0,0.0,25.0,0.0,0.0
4,2020-01-01,PL1331LAHG1S4H,HGST HMS5C4040ALE640,0,0.0,25546.0,NaN,NaN,28.0,0.0,0.0


In [188]:
df.tail()

,date,serial_number,model,failure,smart_5_raw,smart_9_raw,smart_187_raw,smart_188_raw,smart_194_raw,smart_197_raw,smart_198_raw
31945055,2020-12-31,ZJV2ESXT,ST12000NM0007,0,0.0,16158.0,0.0,0.0,21.0,0.0,0.0
31945056,2020-12-31,ZA13ZBCT,ST8000DM002,0,0.0,35836.0,0.0,0.0,36.0,0.0,0.0
31945057,2020-12-31,PL1331LAHGD9NH,HGST HMS5C4040BLE640,0,0.0,33286.0,NaN,NaN,29.0,0.0,0.0
31945058,2020-12-31,PL2331LAHDS4TJ,HGST HMS5C4040BLE640,0,0.0,33298.0,NaN,NaN,29.0,0.0,0.0
31945059,2020-12-31,X0GEV9EC,WDC WUH721414ALE6L4,0,0.0,1543.0,NaN,NaN,40.0,0.0,0.0


Ook kijken wij naar het aantal unieke `date` entries. Dit moet gelijk staan aan 366, gezien 2020 een schrikkeljaar was en daardoor 366 dagen had.

In [189]:
df['date'].drop_duplicates().count()

366

Vervolgens kijken wij nog naar het aantal entries in de sample set.

In [190]:
df.shape

(31945060, 11)

## NaN waardes
In de sample set zitten gegarandeerd NaN waardes. NaN staat voor 'Not a Number'.
In onze sample set willen wij zo min mogelijk NaN waardes hebben, omdat wij met NaN waardes onze predicties niet kunnen uitvoeren.

Ten eerste gaan wij kijken hoeveel NaN waardes elke feature heeft.

In [191]:
df.isnull().sum()

date                   0
serial_number          0
model                  0
failure                0
smart_5_raw        30233
smart_9_raw        30233
smart_187_raw    9942818
smart_188_raw    9942818
smart_194_raw      30233
smart_197_raw      54960
smart_198_raw      30233
dtype: int64

In [192]:
df.isnull().sum().sum()

20061528

Dit komt uiteindelijk uit op een totaal van `20061528` NaN waardes in de hele sample set.

### Procent van missing NaN waardes per feature weergeven
Om een beter beeld te krijgen van de NaN values, maken wij een overzicht met het procent van NaN waardes per feature, ten opzichten van het totaal aantal entries van die feature.

In [193]:
missing_nan = df.isnull().sum() * 100 / len(df)
missing_nan_df = pd.DataFrame({'% NaN': missing_nan})
missing_nan_df.head(15)

,% NaN
date,0.000000
serial_number,0.000000
model,0.000000
failure,0.000000
smart_5_raw,0.094641
smart_9_raw,0.094641
smart_187_raw,31.124744
smart_188_raw,31.124744
smart_194_raw,0.094641
smart_197_raw,0.172045


Uit bovenstaand overzicht kunnen wij concluderen dat de meeste NaN waardes binnen de features `smart_187_raw` en `smart_188_raw` zitten.

### NaN waardes per model
Om een nog beter beeld te krijgen over de NaN waardes, gaan wij een overzicht maken.

Dit overzicht geeft per model, het percentage NaN waardes ten opzichten van het totale aantal entries, per feature weer.

In [194]:
df_nan = df.isnull().groupby(df.model).mean().round(4).mul(100)

In [195]:
df_nan = df_nan.drop(['date', 'serial_number', 'model', 'failure'], axis = 1)
df_nan.head(51)

,smart_5_raw,smart_9_raw,smart_187_raw,smart_188_raw,smart_194_raw,smart_197_raw,smart_198_raw
model,,,,,,,
DELLBOSS VD,100.00,100.00,100.00,100.00,100.00,100.00,100.00
HGST HDS5C4040ALE630,0.00,0.00,100.00,100.00,0.00,0.00,0.00
HGST HMS5C4040ALE640,0.01,0.01,100.00,100.00,0.01,0.01,0.01
HGST HMS5C4040BLE640,0.01,0.01,100.00,100.00,0.01,0.01,0.01
HGST HMS5C4040BLE641,0.00,0.00,100.00,100.00,0.00,0.00,0.00
HGST HUH721010ALE600,0.02,0.02,100.00,100.00,0.02,0.02,0.02
HGST HUH721212ALE600,0.00,0.00,100.00,100.00,0.00,0.00,0.00
HGST HUH721212ALE604,0.23,0.23,100.00,100.00,0.23,0.23,0.23
HGST HUH721212ALN604,0.00,0.00,100.00,100.00,0.00,0.00,0.00


Uit bovenstaand overzicht kunnen wij een aantal dingen concluderen:

- Elk model heeft, of een NaN percentage van onder de 1% (met uitzondering van `WDC WUH721414ALE6L4`, die op 1.13% zit), of een NaN percentage van 100% bij `smart_187_raw` en `smart_188_raw` (met uitzondering van `TOSHIBA MG07ACA14TEY` en `TOSHIBA MG08ACA16TEY`)
- De modellen die 100% NaN hebben bij `smart_187_raw`, hebben ook 100% NaN bij `smart_188_raw`
- Het model `DELLBOSS VD` is geen schijf

Ten eerste halen wij het model `DELLBOSS VD` uit de dataframe, gezien dit geen schijf is.

In [196]:
df = df.drop(df[df.model == 'DELLBOSS VD'].index)

In [197]:
df[df.model == 'DELLBOSS VD'].sum()

date             0.0
serial_number    0.0
model            0.0
failure          0.0
smart_5_raw      0.0
smart_9_raw      0.0
smart_187_raw    0.0
smart_188_raw    0.0
smart_194_raw    0.0
smart_197_raw    0.0
smart_198_raw    0.0
dtype: float64

Ook halen wij `DELLBOSS VD` uit de `df_nan` set.

In [198]:
df_nan = df_nan.drop('DELLBOSS VD')

Vervolgens is het voor ons handig om te zien, hoeveel entries elk model heeft ten opzichte van het totaal aantal entries.

Eerst voegen wij een kolom `amount_entries` toe aan de dataframe.

In [199]:
df_nan.insert(0, "amount_entries", 0)

verder aan werken

## Data opsplitsen
Om de data bruikbaar te maken, splitsen wij deze op in 2 verschillende sets.

Zoals in bovenstaand overzicht te zien is, zijn er veel modellen die alleen `smart_187_raw` en `smart_188_raw` missen. Deze modellen zetten wij in een aparte dataframe.

Ten eerste gaan we kijken welke schijven een NaN percentage van 100% hebben bij `smart_187_raw`. De schijven die een percentage van 100% NaN hebben bij `smart_187_raw`, hebben ook een 100% NaN percentage bij `smart_188_raw`. 

Om dit te doen gebruiken wij de dataframe `df_nan`, die eerder gebruikt is. Hieruit halen wij de modellen die een NaN percentage van 100% hebben.

De entries van deze modellen voegen wij vervolgens toe aan een dataframe `df_2`, en maken nog een dataframe `df_1` aan met alle modellen die niet een NaN percentage van 100% hebben.

In [200]:
df_nan_models = df_nan[df_nan['smart_187_raw'] == 100].index

In [201]:
df_2 = df[df['model'].isin(df_nan_models)]
df_2.shape[0]

(9917391, 11)

In [202]:
df_1 = df[~df['model'].isin(df_nan_models)]
df_1.shape[0]

(22008389, 11)

In [203]:
df.shape[0]

(31925780, 11)

Het totaal aantal entries van de originele dataframe `df` was `31925780`.

Wanneer wij de entries van `df_1` (`22008389`) en `df_2` (`9917391`) optellen, komen wij uit op het volgende:

In [207]:
if (df_1.shape[0] + df_2.shape[0] == df.shape[0]):
    print(True)

else:
    print(False)

True


Dit betekent dat de som van `df_1` en `df_2` gelijk staat aan `df`.

Hieruit kunnen wij concluderen dat bovenstaande code de modellen correct heeft opgesplitst.

## Data exporteren

Uiteindelijk exporteren wij de twee verschillende dataframes naar csv files.

In [205]:
df_1.to_csv('data\\sample_without_100nan.csv')

In [206]:
df_2.to_csv('data\\sample_with_100nan.csv')